In [1]:
# %pip install selenium

In [2]:
#imports
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import re
import time
import tqdm

In [3]:
df=pd.read_csv('Amazon Scraping - Sheet1.csv',index_col= False).drop("Unnamed: 0",axis=1)

In [4]:
def url_formatter(country_code,asin):
    return f"https://www.amazon.{country_code}/dp/{asin}"

In [5]:
url_list=[]
for i in range(len(df)):
    asin=df[['Asin','country']].iloc[i]['Asin']
    country=df[['Asin','country']].iloc[i]['country']
    url_list.append(url_formatter(country,asin))

In [6]:
class AmazonScrapper:
    def __init__(self):
        chrome_options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome('chromedriver',options=chrome_options)
    def quit(self):
        self.driver.quit()
    def get(self,url):
        self.url=url
        self.driver.get(self.url)
    def scrape_page(self):
        self.page_dict={}

        title=self.driver.find_element_by_xpath("//h1[@id='title']").text
        image=self.driver.find_element_by_xpath(f"//img[@alt='{title}']").get_attribute('src')
        price_of_product=self.driver.find_element_by_xpath("//div[@id='corePriceDisplay_desktop_feature_div']//span").text.replace("\n",'.')
#         product_description=self.driver.find_element_by_xpath("//div[@id='productDescription']").text
        product_details={}
        for row in self.driver.find_elements_by_xpath("//div[@id='prodDetails']//table//tbody//tr"):
            th=row.find_element_by_xpath('th').text
            td=row.find_element_by_xpath('td').text
            product_details[th]=td

        self.page_dict['title']=title
        self.page_dict['image']=image
        self.page_dict['price_of_product']=price_of_product
#         self.page_dict['product_description']=product_description
        self.page_dict['product_details']=product_details
        print(f"👍 the {self.url} is available")
        return self.page_dict
    def scrape_page_type2(self):
        self.page_dict={}
        try:
                title=self.driver.find_element_by_xpath("//h1[@id='title']").text            
        except:
            raise Exception("debug: check element of title")
        try:
            image=self.driver.find_element_by_xpath("//img[@id='imgBlkFront']").get_attribute('src')
        except:
            raise Exception("debug: check element of image")
        try:
            try:
                price_of_product=self.driver.find_element_by_xpath("//span[@id='price']").text
            except:
                price_of_product=self.driver.find_element_by_xpath("//span[@class='a-color-base']").text
        except:
            raise Exception("debug: check element of price_of_product")
        try:
            product_details={}
            table=self.driver.find_elements_by_xpath("//div[@id='detailBullets_feature_div']/ul/li")
            for row in table:
                th=row.text.split(":")[0]
                td=row.text.split(":")[1]
                product_details[th]=td
        except:
            raise Exception("debug: check element of product_details")
        self.page_dict['title']=title
        self.page_dict['image']=image
        self.page_dict['price_of_product']=price_of_product
        self.page_dict['product_details']=product_details
        print(f"👍 the {self.url} is available")
        
        return self.page_dict
    def error_status(self):
        try:
            r=re.search(r"\d\d\d",self.driver.find_element_by_xpath("//form").get_attribute("action"))
            if int(r.group()) == 404:
                with open("file.text","a+") as f:
                    f.write("{0} -- {1}\n".format(self.url,"404"))
                    print(f" the {self.url} not available")
                return "404"
        except:
            with open("file.text","a+") as f:
                f.write("{0} -- {1}\n".format( self.url,"invalid"))
                print(f" the {self.url} not available")
            return "invalid"

In [11]:
AmazonScrapper=AmazonScrapper()

In [8]:
# AmazonScrapper.get("https://www.amazon.de/dp/000101742X")

In [9]:
# AmazonScrapper.scrape_page_type2()

In [ ]:
final_dict={}
for url in tqdm.tqdm(url_list):
    
    AmazonScrapper.get(url)
    time.sleep(3)
    try:
        try:
            final_dict[url]=AmazonScrapper.scrape_page_type2()
        except:
            final_dict[url]=AmazonScrapper.scrape_page()    
    except:
        AmazonScrapper.error_status()

  0%|                                                                                 | 1/1000 [00:03<58:07,  3.49s/it]

 the https://www.amazon.de/dp/1015 not available


  0%|▏                                                                              | 2/1000 [00:07<1:00:40,  3.65s/it]

 the https://www.amazon.fr/dp/1015 not available


  0%|▏                                                                              | 3/1000 [00:12<1:14:48,  4.50s/it]

...

 the https://www.amazon.fr/dp/4207181 not available


 90%|██████████████████████████████████████████████████████████████████████▋        | 895/1000 [52:36<06:56,  3.96s/it]

👍 the https://www.amazon.de/dp/000420736X is available


 90%|██████████████████████████████████████████████████████████████████████▊        | 896/1000 [52:41<07:16,  4.20s/it]

👍 the https://www.amazon.fr/dp/000420736X is available


In [ ]:
final_dict

In [ ]:
import json
with open("output.json", "w") as outfile:
    json.dump(final_dict, outfile)

In [ ]:
AmazonScrapper.quit()